In [4]:
%pip install keras
%pip install tensorflow --user

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


  Using cached tensorflow-2.17.0-cp312-cp312-win_amd64.whl.metadata (3.2 kB)
  Using cached tensorflow_intel-2.17.0-cp312-cp312-win_amd64.whl.metadata (5.0 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl.metadata (4.4 kB)
  Using cached gast-0.6.0-py3-none-any.whl.metadata (1.3 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl.metadata (814 bytes)
  Using cached tensorboard-2.17.1-py3-none-any.whl.metadata (1.6 kB)
Using cached tensorflow-2.17.0-cp312-cp312-win_amd64.whl (2.0 kB)
Using cached tensorflow_intel-2.17.0-cp312-cp312-win_amd64.whl (385.2 MB)
Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
Using cached gast-0.6.0-py3-none-any.whl (21 kB)
Using cached google_pasta-0.2.0-py3-none-any.whl (57 kB)
Using cached tensorboard-2.17.1-py3-none-any.whl (5.5 MB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
!kaggle datasets download -d vishalmane109/indian-currency-note-images-dataset-2020
!unzip indian-currency-note-images-dataset-2020.zip

'kaggle' is not recognized as an internal or external command,
operable program or batch file.
'unzip' is not recognized as an internal or external command,
operable program or batch file.


In [4]:
import os
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Correct absolute paths for the dataset directories
train_dir = r"F:\Currency_Detection\venv\Indian currency dataset v1\training" 
validation_dir = r"F:\Currency_Detection\venv\Indian currency dataset v1\validation"  
test_dir = r"F:\Currency_Detection\venv\Indian currency dataset v1\test"  

# ImageDataGenerator for training and validation sets
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255.0,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

validation_datagen = ImageDataGenerator(rescale=1.0 / 255.0)

# Train generator
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical'
)

# Validation generator
validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical'
)

# Create a DataFrame for test images
test_images = [os.path.join(test_dir, fname) for fname in os.listdir(test_dir) if fname.endswith(('png', 'jpg', 'jpeg'))]
test_df = pd.DataFrame({'filename': test_images})

# Use ImageDataGenerator for test set without labels
test_datagen = ImageDataGenerator(rescale=1.0 / 255.0)

# Test generator using flow_from_dataframe
test_generator = test_datagen.flow_from_dataframe(
    test_df,
    x_col='filename',
    y_col=None,  # No labels for test
    target_size=(128, 128),
    batch_size=32,
    class_mode=None,  # No class mode for test data
    shuffle=False  # Don't shuffle test data
)

# Now you can use test_generator for predictions


Found 3566 images belonging to 8 classes.
Found 345 images belonging to 8 classes.
Found 91 validated image filenames.


In [6]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Load the MobileNetV2 model without the top layers
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(128, 128, 3))

# Add custom layers on top of the base model
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)  # You can adjust the number of neurons
predictions = Dense(len(train_generator.class_indices), activation='softmax')(x)

# Create the model
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Callbacks for training
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
model_checkpoint = ModelCheckpoint("best_model.keras", save_best_only=True)  # Change to .keras

# Fit the model (make sure to set the epochs and steps)
history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=20,  # Adjust the number of epochs as needed
    callbacks=[early_stopping, model_checkpoint]
)


Epoch 1/20


c:\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


112/112 ━━━━━━━━━━━━━━━━━━━━ 218s 2s/step - accuracy: 0.4769 - loss: 1.6661 - val_accuracy: 0.6783 - val_loss: 0.9445
Epoch 2/20
112/112 ━━━━━━━━━━━━━━━━━━━━ 89s 756ms/step - accuracy: 0.7282 - loss: 0.7942 - val_accuracy: 0.6928 - val_loss: 0.8428
Epoch 3/20
112/112 ━━━━━━━━━━━━━━━━━━━━ 109s 937ms/step - accuracy: 0.7735 - loss: 0.6347 - val_accuracy: 0.7362 - val_loss: 0.7633
Epoch 4/20
112/112 ━━━━━━━━━━━━━━━━━━━━ 80s 674ms/step - accuracy: 0.7776 - loss: 0.6211 - val_accuracy: 0.7043 - val_loss: 0.8854
Epoch 5/20
112/112 ━━━━━━━━━━━━━━━━━━━━ 79s 671ms/step - accuracy: 0.8150 - loss: 0.5159 - val_accuracy: 0.7246 - val_loss: 0.7509
Epoch 6/20
112/112 ━━━━━━━━━━━━━━━━━━━━ 88s 747ms/step - accuracy: 0.8195 - loss: 0.4896 - val_accuracy: 0.7130 - val_loss: 0.8457
Epoch 7/20
112/112 ━━━━━━━━━━━━━━━━━━━━ 80s 676ms/step - accuracy: 0.8383 - loss: 0.4629 - val_accuracy: 0.7420 - val_loss: 0.7391
Epoch 8/20
112/112 ━━━━━━━━━━━━━━━━━━━━ 79s 675ms/step - accuracy: 0.8668 - loss: 0.4033 - val_

In [8]:
# Unfreeze some of the base model layers
for layer in base_model.layers[-50:]:
    layer.trainable = True

# Re-compile the model with a lower learning rate
from tensorflow.keras.optimizers import Adam
model.compile(optimizer=Adam(learning_rate=1e-5), loss='categorical_crossentropy', metrics=['accuracy'])

# Fine-tune the model
history_fine_tune = model.fit(
    train_generator,
    epochs=20,
    validation_data=validation_generator,
    callbacks=[early_stopping, model_checkpoint]  
)

# Evaluate the model on validation data
val_loss, val_accuracy = model.evaluate(validation_generator)
print(f"Validation Accuracy: {val_accuracy * 100:.2f}%")


Epoch 1/20
112/112 ━━━━━━━━━━━━━━━━━━━━ 100s 751ms/step - accuracy: 0.5771 - loss: 1.6694 - val_accuracy: 0.7420 - val_loss: 0.8440
Epoch 2/20
112/112 ━━━━━━━━━━━━━━━━━━━━ 86s 729ms/step - accuracy: 0.7391 - loss: 0.8046 - val_accuracy: 0.7391 - val_loss: 0.9073
Epoch 3/20
112/112 ━━━━━━━━━━━━━━━━━━━━ 86s 726ms/step - accuracy: 0.7874 - loss: 0.6520 - val_accuracy: 0.7710 - val_loss: 0.7949
Epoch 4/20
112/112 ━━━━━━━━━━━━━━━━━━━━ 87s 736ms/step - accuracy: 0.7978 - loss: 0.6016 - val_accuracy: 0.7855 - val_loss: 0.7211
Epoch 5/20
112/112 ━━━━━━━━━━━━━━━━━━━━ 87s 741ms/step - accuracy: 0.8331 - loss: 0.5172 - val_accuracy: 0.8087 - val_loss: 0.6831
11/11 ━━━━━━━━━━━━━━━━━━━━ 7s 658ms/step - accuracy: 0.7528 - loss: 0.8793
Validation Accuracy: 74.20%


In [10]:
model.save('my_model.keras')